In [33]:
import os
import json

dataLists = []

path = './data/interaction_ros_logs/pilot/session1'
for f in os.listdir(path):
    if os.path.isfile(os.path.join(path,f)):
        
        # Read in file
        try:
            fin = open(os.path.join(path,f),'r')
            data = json.load(fin)
            fin.close()
        except:
            print('Error reading file {}'.format(f))

        # extract raw time trace
        traceStarted = False
        rawTrace = []
        for key in sorted(data.keys()):

            # find start of trace
            if data[key]['event'] == 'start' and len(data[key]['timeline']) > 1:
                traceStarted = True
                rawTrace.append(float(key))
                #print(data[key])
                continue

            # end of trace
            if traceStarted and data[key]['event'] == 'stop':
                rawTrace.append(float(key))
                #print(data[key])
                break

            # append time data when in trace
            if traceStarted and data[key]['event'] == 'sync' and data[key]['index'] != 0:
                rawTrace.append(float(key))
                #print(data[key])

        # convert into trace offset by start time 
        offsetTrace = [n-rawTrace[0] for n in rawTrace]

        # convert into durations per step
        durations = [rawTrace[i] - rawTrace[i-1] for i in range(1,len(rawTrace))]
        
         # Fill in name fields
        fields = f.split('_')
        participant = int(fields[0].replace('p',''))
        trial = int(fields[2].replace('t',''))
        row = [participant,trial] + durations
        
        print('Data',row)
        #print('Length',len(row))
        print('____________________________________________')
        
        dataLists.append(row)

Data [1, 1, 3.531599998474121, 13.930000066757202, 32.59000015258789, 4.129999876022339, 27.610000133514404, 5.609999895095825, 34.609999895095825, 7.490000009536743, 24.100000143051147, 6.0299999713897705, 22.08999991416931, 8.110000133514404]
____________________________________________
Data [1, 2, 3.506200075149536, 2.109999895095825, 32.610000133514404, 7.509999990463257, 27.59999990463257, 5.990000009536743, 34.61999988555908, 5.509999990463257, 24.100000143051147, 3.8899998664855957, 22.09000015258789, 1.0]
____________________________________________
Data [1, 3, 3.5199999809265137, 2.1399998664855957, 32.62000012397766, 4.6499998569488525, 27.59000015258789, 6.549999952316284, 34.609999895095825, 5.010000228881836, 24.08999991416931, 3.009999990463257, 22.08999991416931, 1.320000171661377]
____________________________________________
Data [2, 1, 3.5299999713897705, 11.639999866485596, 32.610000133514404, 14.599999904632568, 27.610000133514404, 11.0, 34.59999990463257, 13.3499999

In [34]:
import pandas as pd

header = ['PID','Trial'] + ['Duration-{}'.format(i) for i in range(1,13)]
df = pd.DataFrame(dataLists, columns=header)

In [35]:
df

,PID,Trial,Duration-1,Duration-2,Duration-3,Duration-4,Duration-5,Duration-6,Duration-7,Duration-8,Duration-9,Duration-10,Duration-11,Duration-12
0,1,1,3.5316,13.93,32.59,4.13,27.61,5.61,34.61,7.49,24.10,6.03,22.09,8.11
1,1,2,3.5062,2.11,32.61,7.51,27.60,5.99,34.62,5.51,24.10,3.89,22.09,1.00
2,1,3,3.5200,2.14,32.62,4.65,27.59,6.55,34.61,5.01,24.09,3.01,22.09,1.32
3,2,1,3.5300,11.64,32.61,14.60,27.61,11.00,34.60,13.35,24.10,9.96,22.10,22.21
4,2,2,3.5400,34.05,32.60,7.90,27.60,7.74,34.63,17.12,24.10,5.38,22.09,4.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,34,2,3.5300,7.18,32.63,14.27,27.61,9.95,34.61,15.61,24.11,11.56,22.12,7.25
101,34,3,3.5387,11.26,32.62,5.68,27.61,3.17,34.62,3.13,24.11,1.18,22.11,5.12
102,35,1,3.5100,4.70,32.62,7.14,27.62,11.96,34.63,12.16,24.12,23.81,22.10,8.39
103,35,2,3.5200,2.60,32.63,5.05,27.61,11.56,34.64,8.24,24.12,4.08,22.13,6.51


In [36]:
df.to_csv('./data/interaction_ros_logs/pilot_results.csv', index=False, header=True)